## Load features, select columns, prep targets

- Goal: load enriched data and select a minimal, stable feature set.
- We’ll start small with interpretable features and fill small gaps with medians.
- Targets: finish_pos (regression) and top10 (classification).

In [1]:
import pandas as pd, numpy as np
from pathlib import Path

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PROCESSED = ROOT / "data" / "processed"
df = pd.read_csv(PROCESSED / "season_2023_features_enriched.csv")

if "grid_drop" not in df.columns and {"grid_pos","qual_pos"}.issubset(df.columns):
    df["grid_drop"] = df["grid_pos"] - df["qual_pos"]

features = [c for c in [
    "fp_mean_all_s","fp_median_longrun_s","fp_total_laps",
    "qual_pos","delta_to_pole_s","grid_pos","grid_drop",
    "roll3_avg_pos","roll5_avg_pos","roll3_pts","roll5_pts","roll5_dnf_rate",
    "team_roll3_pts","team_roll5_pts","track_hist_avg_pos",
] if c in df.columns]

X = df[features].fillna(df[features].median(numeric_only=True))
y_reg = df["finish_pos"].astype(float)
y_cls = df["top10"].astype(int)
groups = df["group_key"]
print("Features:", features)
print("Rows:", X.shape[0], "| groups:", groups.nunique())


Features: ['fp_mean_all_s', 'fp_median_longrun_s', 'fp_total_laps', 'qual_pos', 'delta_to_pole_s', 'grid_pos', 'grid_drop', 'roll3_avg_pos', 'roll5_avg_pos', 'roll3_pts', 'roll5_pts', 'roll5_dnf_rate', 'team_roll3_pts', 'team_roll5_pts', 'track_hist_avg_pos']
Rows: 200 | groups: 10


## Diagnose missing & constant features

- Goal: see where the NaNs/∞ and constants are before modeling.
- This shows which columns have missing values and which are constant (no variance).
- Constant or all-NaN columns don’t help models and can break scalers/imputers.

In [2]:
import numpy as np

nan_counts = X.isna().sum().sort_values(ascending=False)
const_cols = [c for c in X.columns if X[c].nunique(dropna=True) <= 1]
print("Top NaN counts:\n", nan_counts.head(10))
print("Constant columns:", const_cols[:10], "… total:", len(const_cols))


Top NaN counts:
 track_hist_avg_pos     200
fp_median_longrun_s      0
fp_mean_all_s            0
qual_pos                 0
delta_to_pole_s          0
grid_pos                 0
fp_total_laps            0
grid_drop                0
roll3_avg_pos            0
roll3_pts                0
dtype: int64
Constant columns: ['roll5_dnf_rate', 'track_hist_avg_pos'] … total: 2


## Cleaning X: drop all-NaN & constant cols, replace ±∞

- Goal: make features model-friendly; keep column list for later.
- We safely remove unusable columns so the imputer/scaler has real data to work with.
- We keep the final feature list to use for training and any coefficient peek.

In [3]:
from copy import deepcopy

Xc = X.replace([np.inf, -np.inf], np.nan).copy()

# Drop columns that are entirely NaN
all_nan_cols = Xc.columns[Xc.isna().all()]
if len(all_nan_cols):
    print("Dropping all-NaN columns:", list(all_nan_cols))
    Xc = Xc.drop(columns=list(all_nan_cols))

# Drop columns that are constant (no variance)
const_cols = [c for c in Xc.columns if Xc[c].nunique(dropna=True) <= 1]
if len(const_cols):
    print("Dropping constant columns:", const_cols)
    Xc = Xc.drop(columns=const_cols)

features_clean = Xc.columns.tolist()
print("Kept", len(features_clean), "features.")


Dropping all-NaN columns: ['track_hist_avg_pos']
Dropping constant columns: ['roll5_dnf_rate']
Kept 13 features.


## GroupKFold baselines

- Goal: add SimpleImputer(median) so Linear/Logistic can handle missing values.
- SimpleImputer(median) resolves the NaN error; the scaler handles different scales.
- We print concise metrics for both tasks using GroupKFold (no weekend leakage).

In [4]:
# Goal: add SimpleImputer(median) so Linear/Logistic can handle missing values.
import numpy as np, pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score

def spearman_like(y_true, y_pred):
    a = pd.Series(y_true).rank()
    b = pd.Series(y_pred).rank()
    return a.corr(b)

gkf = GroupKFold(n_splits=min(5, groups.nunique()))
mae, spr, acc, f1, brier = [], [], [], [], []

for tr, te in gkf.split(Xc, y_reg, groups):
    Xtr, Xte = Xc.iloc[tr], Xc.iloc[te]
    ytr_r, yte_r = y_reg.iloc[tr], y_reg.iloc[te]
    ytr_c, yte_c = y_cls.iloc[tr], y_cls.iloc[te]

    # Regression (imputer + scaler + linear reg)
    reg = make_pipeline(SimpleImputer(strategy="median"),
                        StandardScaler(with_mean=False),
                        LinearRegression())
    reg.fit(Xtr, ytr_r)
    pred_r = reg.predict(Xte)
    mae.append(mean_absolute_error(yte_r, pred_r))
    spr.append(spearman_like(yte_r, pred_r))

    # Classification (imputer + scaler + logistic)
    clf = make_pipeline(SimpleImputer(strategy="median"),
                        StandardScaler(with_mean=False),
                        LogisticRegression(max_iter=1000, solver="liblinear"))
    clf.fit(Xtr, ytr_c)
    proba = clf.predict_proba(Xte)[:, 1]
    pred_c = (proba >= 0.5).astype(int)
    acc.append(accuracy_score(yte_c, pred_c))
    f1.append(f1_score(yte_c, pred_c))
    brier.append(np.mean((proba - yte_c.values)**2))

print(f"Regression — MAE: {np.mean(mae):.2f} | Spearman: {np.mean(spr):.3f}")
print(f"Classification — Acc: {np.mean(acc):.2f} | F1: {np.mean(f1):.2f} | Brier: {np.mean(brier):.3f}")

Regression — MAE: 2.28 | Spearman: -0.071
Classification — Acc: 0.86 | F1: 0.86 | Brier: 0.099


## Refit on all data & view coefficients

- Goal: fit one final regression on all rows and inspect coefficients
- This gives a rough sense of which features move predicted finishing position.
- We’ll add tree-based importances in the “stronger models” step next.

In [5]:
final_reg = make_pipeline(SimpleImputer(strategy="median"),
                          StandardScaler(with_mean=False),
                          LinearRegression())
final_reg.fit(Xc, y_reg)

coefs = pd.Series(final_reg.named_steps["linearregression"].coef_, index=features_clean).sort_values()
print("Top +ve:\n", coefs.tail(5), "\n\nTop -ve:\n", coefs.head(5))


Top +ve:
 qual_pos          0.573772
roll5_pts         0.680658
team_roll5_pts    1.029664
roll5_avg_pos     1.330264
roll3_avg_pos     3.138200
dtype: float64 

Top -ve:
 team_roll3_pts   -1.180159
roll3_pts        -0.232500
fp_mean_all_s    -0.161033
grid_drop        -0.148764
fp_total_laps    -0.000673
dtype: float64


## RandomForest (regression) with small param sweep + GroupKFold

- Goal: try a tiny RF regressor grid with GroupKFold and pick the best by MAE.
- We evaluate a tiny RF grid with GroupKFold and choose the params that minimize MAE.
- Spearman checks ranking quality; we keep the best config for final fit.

In [6]:
import numpy as np, pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

param_grid = [
    {"n_estimators": 300, "max_depth": None, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": 12,   "min_samples_leaf": 2},
    {"n_estimators": 500, "max_depth": 16,   "min_samples_leaf": 2},
]

gkf = GroupKFold(n_splits=min(5, groups.nunique()))
best, best_mae, best_spr = None, 1e9, None

def spearman_like(y_true, y_pred):
    a, b = pd.Series(y_true).rank(), pd.Series(y_pred).rank()
    return a.corr(b)

for p in param_grid:
    maes, sprs = [], []
    for tr, te in gkf.split(Xc, y_reg, groups):
        imputer = SimpleImputer(strategy="median")
        Xtr, Xte = imputer.fit_transform(Xc.iloc[tr]), imputer.transform(Xc.iloc[te])
        ytr, yte = y_reg.iloc[tr], y_reg.iloc[te]
        rf = RandomForestRegressor(random_state=42, n_jobs=-1, **p)
        rf.fit(Xtr, ytr)
        pred = rf.predict(Xte)
        maes.append(mean_absolute_error(yte, pred))
        sprs.append(spearman_like(yte, pred))
    m_mae, m_spr = float(np.mean(maes)), float(np.mean(sprs))
    print("RF-Reg", p, "→ MAE:", f"{m_mae:.2f}", "| Spearman:", f"{m_spr:.3f}")
    if m_mae < best_mae:
        best, best_mae, best_spr = p, m_mae, m_spr

print("Best RF-Reg:", best, "| MAE:", f"{best_mae:.2f}", "| Spearman:", f"{best_spr:.3f}")


RF-Reg {'n_estimators': 300, 'max_depth': None, 'min_samples_leaf': 1} → MAE: 2.34 | Spearman: -0.108
RF-Reg {'n_estimators': 300, 'max_depth': 12, 'min_samples_leaf': 2} → MAE: 2.33 | Spearman: -0.102
RF-Reg {'n_estimators': 500, 'max_depth': 16, 'min_samples_leaf': 2} → MAE: 2.31 | Spearman: -0.114
Best RF-Reg: {'n_estimators': 500, 'max_depth': 16, 'min_samples_leaf': 2} | MAE: 2.31 | Spearman: -0.114


## RandomForest (classification) + GroupKFold (Acc/F1/Brier)

- Goal: tiny RF classifier grid with GroupKFold, pick best by Brier (prob quality).
- We select the classifier by lowest Brier (best-calibrated probabilities), and report Acc/F1 too.
This gives us a stronger points-probability model for simulation.

In [7]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

param_grid_cls = [
    {"n_estimators": 300, "max_depth": None, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": 10,   "min_samples_leaf": 2},
    {"n_estimators": 500, "max_depth": 14,   "min_samples_leaf": 2},
]

gkf = GroupKFold(n_splits=min(5, groups.nunique()))
best_c, best_brier, best_tuple = None, 1e9, None

for p in param_grid_cls:
    accs, f1s, briers = [], [], []
    for tr, te in gkf.split(Xc, y_cls, groups):
        imp = SimpleImputer(strategy="median")
        Xtr, Xte = imp.fit_transform(Xc.iloc[tr]), imp.transform(Xc.iloc[te])
        ytr, yte = y_cls.iloc[tr], y_cls.iloc[te]
        rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight=None, **p)
        rf.fit(Xtr, ytr)
        proba = rf.predict_proba(Xte)[:, 1]
        pred = (proba >= 0.5).astype(int)
        accs.append(accuracy_score(yte, pred))
        f1s.append(f1_score(yte, pred))
        briers.append(np.mean((proba - yte.values)**2))
    m_acc, m_f1, m_brier = float(np.mean(accs)), float(np.mean(f1s)), float(np.mean(briers))
    print("RF-Cls", p, "→ Acc:", f"{m_acc:.2f}", "| F1:", f"{m_f1:.2f}", "| Brier:", f"{m_brier:.3f}")
    if m_brier < best_brier:
        best_c, best_brier, best_tuple = p, m_brier, (m_acc, m_f1)

print("Best RF-Cls:", best_c, "| Acc:", f"{best_tuple[0]:.2f}", "| F1:", f"{best_tuple[1]:.2f}", "| Brier:", f"{best_brier:.3f}")


RF-Cls {'n_estimators': 300, 'max_depth': None, 'min_samples_leaf': 1} → Acc: 0.84 | F1: 0.84 | Brier: 0.114
RF-Cls {'n_estimators': 300, 'max_depth': 10, 'min_samples_leaf': 2} → Acc: 0.83 | F1: 0.83 | Brier: 0.111
RF-Cls {'n_estimators': 500, 'max_depth': 14, 'min_samples_leaf': 2} → Acc: 0.83 | F1: 0.83 | Brier: 0.111
Best RF-Cls: {'n_estimators': 500, 'max_depth': 14, 'min_samples_leaf': 2} | Acc: 0.83 | F1: 0.83 | Brier: 0.111


## Fit best RFs on all data and save

- Goal:refit best RF models on all rows and save them for reuse.
- We train both best models on all data and save them to /models for reuse.
- These will power the Monte Carlo simulation in the next phase.

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from joblib import dump
from pathlib import Path

MODELS = (Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()) / "models"
MODELS.mkdir(exist_ok=True)

# Refit regression
imp_reg = SimpleImputer(strategy="median")
rf_reg  = RandomForestRegressor(random_state=42, n_jobs=-1, **best)
reg_pipe = Pipeline([("imp", imp_reg), ("rf", rf_reg)])
reg_pipe.fit(Xc, y_reg)
dump(reg_pipe, MODELS / "rf_finishpos.joblib")

# Refit classification
imp_cls = SimpleImputer(strategy="median")
rf_cls  = RandomForestClassifier(random_state=42, n_jobs=-1, **best_c)
cls_pipe = Pipeline([("imp", imp_cls), ("rf", rf_cls)])
cls_pipe.fit(Xc, y_cls)
dump(cls_pipe, MODELS / "rf_top10.joblib")

print("Saved:", (MODELS / 'rf_finishpos.joblib').name, "and", (MODELS / 'rf_top10.joblib').name)


Saved: rf_finishpos.joblib and rf_top10.joblib
